In [19]:
import pandas as pd

# Load the new data
cleaned_data = pd.read_csv('7_cleaned_trips.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [20]:
# Function to convert time string to timedelta, adjusting for 24 hour+ times
def convert_time(time_str):
    # Remove the '0 days ' part of the string
    time_str = time_str.replace('0 days ', '')
    h, m, s = map(int, time_str.split(':'))
    if h >= 24:
        h = h - 24
    return pd.to_timedelta(f'{h}:{m}:{s}')



# Convert 'arrival_time_new' to timedelta format
cleaned_data['arrival_time_new'] = cleaned_data['arrival_time_new'].apply(convert_time)

# Group by 'trip_id' and sort within each group by 'stop_sequence'
grouped_cleaned = cleaned_data.sort_values('stop_sequence').groupby('trip_id')

# Create a new dataframe to hold the first stop of each trip
first_stops_cleaned = grouped_cleaned.first().reset_index()

# Create an additional column for time in hours
first_stops_cleaned['arrival_time_hours'] = first_stops_cleaned['arrival_time_new'].dt.seconds / 3600

# Define time categories in hours
bins = [0, 4, 10, 16, 22]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']

# Assign these categories to a new 'time_category' column
first_stops_cleaned['time_category'] = pd.cut(first_stops_cleaned['arrival_time_hours'], bins=bins, labels=labels, include_lowest=True, ordered=False)

# For times between 22:00 and 24:00, assign 'Night'
first_stops_cleaned.loc[first_stops_cleaned['arrival_time_hours'] >= 22, 'time_category'] = 'Night'

# Merge 'time_category' column from 'first_stops_cleaned' back to 'cleaned_data'
cleaned_data = pd.merge(cleaned_data, first_stops_cleaned[['trip_id', 'time_category']], on='trip_id', how='left')

In [21]:
cleaned_data.to_csv("8_with_time_bins.csv",index=False)

In [22]:
# Count unique 'trip_id' for each 'day_of_the_week'
unique_trip_ids = cleaned_data.groupby('day_of_the_week')['trip_id'].nunique()

print('Number of unique trip ids for each day of the week:\n', unique_trip_ids)

Number of unique trip ids for each day of the week:
 day_of_the_week
Friday       12
Monday       26
Saturday     46
Sunday       60
Thursday     21
Tuesday      21
Wednesday    22
Name: trip_id, dtype: int64


In [23]:
# Sort the dataframe by 'trip_id' and 'stop_sequence'
cleaned_data = cleaned_data.sort_values(by=['trip_id', 'stop_sequence'])

# Convert 'arrival_time_new' to seconds
cleaned_data['arrival_time_seconds'] = cleaned_data['arrival_time_new'].dt.total_seconds()

# Calculate the time difference between stops within each trip
cleaned_data['time_diff'] = cleaned_data.groupby('trip_id')['arrival_time_seconds'].diff()

# Shift the 'time_diff' values up by one row within each trip
cleaned_data['time_diff'] = cleaned_data.groupby('trip_id')['time_diff'].shift(-1)

# Replace NaN values in 'time_diff' with 0 (for the last stop of each trip)
cleaned_data['time_diff'] = cleaned_data['time_diff'].fillna(0)

# Display the first few rows of the dataframe
cleaned_data.head()

,Unnamed: 0,trip_id,arrival_time,departure_time,stop_sequence,shape_dist_traveled,arrival_time_new,day_of_the_week,time_category,arrival_time_seconds,time_diff
121,121.0,15351.y1002.60-145-b12-1.330.I,10:00:00,10:00:00,1,0.00,0 days 10:00:00,Saturday,Morning,36000.0,138.0
122,122.0,15351.y1002.60-145-b12-1.330.I,10:02:18,10:02:18,2,448.42,0 days 10:02:18,Saturday,Morning,36138.0,34.0
123,123.0,15351.y1002.60-145-b12-1.330.I,10:02:52,10:02:52,3,696.52,0 days 10:02:52,Saturday,Morning,36172.0,67.0
124,124.0,15351.y1002.60-145-b12-1.330.I,10:03:59,10:03:59,4,1187.07,0 days 10:03:59,Saturday,Morning,36239.0,58.0
125,125.0,15351.y1002.60-145-b12-1.330.I,10:04:57,10:04:57,5,1607.52,0 days 10:04:57,Saturday,Morning,36297.0,57.0


In [24]:
cleaned_data.to_csv("9_with_time_diff.csv",index=False)

In [26]:
# Count unique 'trip_id' for each 'day_of_the_week'
unique_trip_ids = cleaned_data.groupby('day_of_the_week')['trip_id'].nunique()

print('Number of unique trip ids for each day of the week:\n', unique_trip_ids)

Number of unique trip ids for each day of the week:
 day_of_the_week
Friday       12
Monday       26
Saturday     46
Sunday       60
Thursday     21
Tuesday      21
Wednesday    22
Name: trip_id, dtype: int64


In [27]:
# Shift the 'stop_sequence' and 'shape_dist_traveled' columns up by one row within each trip
cleaned_data['next_stop_sequence'] = cleaned_data.groupby('trip_id')['stop_sequence'].shift(-1)
cleaned_data['next_shape_dist_traveled'] = cleaned_data.groupby('trip_id')['shape_dist_traveled'].shift(-1)

# For the last stop of each trip, set 'next_stop_sequence' to the same value as 'stop_sequence'
cleaned_data['next_stop_sequence'] = cleaned_data['next_stop_sequence'].fillna(cleaned_data['stop_sequence'])

# Calculate the distance to the next stop
cleaned_data['dist_to_next_stop'] = cleaned_data['next_shape_dist_traveled'] - cleaned_data['shape_dist_traveled']

# For the last stop of each trip, set 'dist_to_next_stop' to 0
cleaned_data['dist_to_next_stop'] = cleaned_data['dist_to_next_stop'].fillna(0)

# Display the first few rows of the dataframe
cleaned_data.head()


,Unnamed: 0,trip_id,arrival_time,departure_time,stop_sequence,shape_dist_traveled,arrival_time_new,day_of_the_week,time_category,arrival_time_seconds,time_diff,next_stop_sequence,next_shape_dist_traveled,dist_to_next_stop
121,121.0,15351.y1002.60-145-b12-1.330.I,10:00:00,10:00:00,1,0.00,0 days 10:00:00,Saturday,Morning,36000.0,138.0,2.0,448.42,448.42
122,122.0,15351.y1002.60-145-b12-1.330.I,10:02:18,10:02:18,2,448.42,0 days 10:02:18,Saturday,Morning,36138.0,34.0,3.0,696.52,248.10
123,123.0,15351.y1002.60-145-b12-1.330.I,10:02:52,10:02:52,3,696.52,0 days 10:02:52,Saturday,Morning,36172.0,67.0,4.0,1187.07,490.55
124,124.0,15351.y1002.60-145-b12-1.330.I,10:03:59,10:03:59,4,1187.07,0 days 10:03:59,Saturday,Morning,36239.0,58.0,5.0,1607.52,420.45
125,125.0,15351.y1002.60-145-b12-1.330.I,10:04:57,10:04:57,5,1607.52,0 days 10:04:57,Saturday,Morning,36297.0,57.0,6.0,2090.99,483.47


In [28]:
cleaned_data.to_csv("10_training.csv")